# File-level dependency table

| Module / file     | Depends on (internal)                                                      | Depends on (external)                            | Exports / main classes & functions (most relevant)                                                                    | Purpose / notes                                                                                                                                              |
| ----------------- | -------------------------------------------------------------------------- | ------------------------------------------------ | --------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `__init__.py`     | `nn`, `fields`, `solver`, `interpolator`, `meshing`, `svh`, `utils`, `ext` | `torch`, `numpy` (typical)                       | **`Reconstructor`** (high-level class), convenience factory functions; re-exports core types like `Field`             | The main user entry point (e.g., `reconstructor = nksr.Reconstructor(device)`). Orchestrates network inference → field build → meshing. ([GitHub][1])        |
| `nn/` (the “net”) | `utils`                                                                    | `torch`, `torch.nn`, `torch.nn.functional`       | Encoders / UNet-like backbones; voxel feature & normal heads                                                          | Defines the neural nets that predict sparse voxel features & normals used by kernels.                                                                        |
| `fields/`         | `interpolator`, `svh`, `utils`, possibly `ext`                             | `torch`, `numpy`                                 | **`Field`** (implicit surface representation), `Field.evaluate`, **`Field.extract_dual_mesh`** (delegates to meshing) | Wraps learned kernel field; provides evaluation and mesh extraction hooks. The export name `extract_dual_mesh` is shown in the README example. ([GitHub][1]) |
| `svh.py`          | `utils`, `ext`, maybe `interpolator`                                       | —                                                | `SparseVoxelHierarchy` (SVH), builders/helpers                                                                        | Sparse Voxel Hierarchy structure for features, indices, neighborhood queries. Backed by C/C++ ops via `_C.so`.                                               |
| `interpolator.py` | `utils`                                                                    | `torch`                                          | `Interpolator` / `TrilinearInterpolator` helpers                                                                      | Interpolates features/normals on the sparse voxel grid.                                                                                                      |
| `solver.py`       | `utils`, `fields`, possibly `ext`                                          | `torch`, `scipy.sparse` (typical), or custom ops | Linear-system routines (e.g., conjugate gradient); kernel Gram matrix assembly                                        | Solves for per-voxel coefficients α in the NKSR kernel system.                                                                                               |
| `meshing.py`      | `fields`, `utils`, `ext`                                                   | `torch` / `numpy`                                | `extract_dual_mesh`, marching-cubes/dual-contouring driver                                                            | Extracts meshes from field values on voxel corners (dual marching cubes flow). The README calls `field.extract_dual_mesh(...)`. ([GitHub][1])                |
| `ext.py`          | `_C.so`                                                                    | —                                                | Python bindings / thin wrappers                                                                                       | Loads the compiled extension and exposes fast kernels (SVH ops, neighborhood, marching cubes acceleration).                                                  |
| `_C.so` (binary)  | —                                                                          | —                                                | Native ops used by `ext.py`                                                                                           | Compiled CUDA/C++ extension with performance-critical pieces.                                                                                                |
| `configs.py`      | —                                                                          | `yaml` / stdlib                                  | Config dataclasses/utilities                                                                                          | Small configuration helpers for default params and toggles.                                                                                                  |
| `utils.py`        | —                                                                          | `numpy`, `torch`, stdlib                         | IO helpers, point-cloud utils, timing/logging                                                                         | Glue utilities used across modules.                                                                                                                          |

[1]: https://github.com/nv-tlabs/NKSR "GitHub - nv-tlabs/NKSR: [CVPR 2023 Highlight] Neural Kernel Surface Reconstruction"
